<a href="https://colab.research.google.com/github/MLGlobalHealth/StatML4PopHealth/blob/main/lectures/day3/hsgp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scalable Gaussian Process Regression with Stan

## Objectives
In this practical, you will learn about how to implement an fast and scalable Gaussian Process (GP) approximation in Stan. More specificially, this practical will teach you how to implement the Hilbert space approximate Gaussian process (HSGP) proposed by Arno Solin and Simo Sarkkar in 2020 [1] and picked up by Riutort-Mayol et al. (2022) [2] for use in probabilistic programming frameworks such as Stan.

1. Arno Solin and Simo Sarkka (2020). Hilber space methods for reduced-rank Gaussian process regression. *Statistics and Computing*.
2. Gabriel Riutort-Mayol et al. (2022), Practical Hilbert space approximate Gaussian processes for probabilistic programming. *Statistics and Computing*.

By the end of this practical,
1. You will have a better understanding of implementing custom functions in Stan;
2. You will improve your ability to translate mathematics into Stan code;
3. You will have a better understanding of HSGP and its implementation in Stan.

## Flow of the practical
1. Review of Hilbert Space approximate Gaussian Processes
2. How to implement HSGP in Stan
3. An application of GPs for causal inference

In [ ]:
# Install CmdStanPy for Google Colab
!curl -O "https://raw.githubusercontent.com/MLGlobalHealth/StatML4PopHealth/main/practicals/resources/scripts/utilities.py"
from utilities import custom_install_cmdstan, test_cmdstan_installation
custom_install_cmdstan()

In [ ]:
import numpy as np
import pandas as pd

from cmdstanpy import CmdStanModel
import arviz as az

import matplotlib.pyplot as plt
import seaborn as sns

# Visualisation defaults
sns.set_theme(style='whitegrid')
plt.rc('font', size=9)          # controls default text sizes
plt.rc('axes', titlesize=10)    # fontsize of the axes title
plt.rc('axes', labelsize=9)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=9)    # fontsize of the tick labels
plt.rc('ytick', labelsize=9)    # fontsize of the tick labels
plt.rc('legend', fontsize=9)    # legend fontsize

## The dataset
For this tutorial, we will use the `nile` dataset available via the `statsmodels` library. The dataset consists of annual flow measurements of the Nile River at Aswan from 1871 to 1970.

In [ ]:
from statsmodels.datasets import nile
nile = nile.load_pandas().data

# Plot the data
fig, ax = plt.subplots()
nile.plot(x='year', y='volume', ax=ax)
ax.set_title('Nile River Volume')
ax.set_xlabel('Year')
ax.set_ylabel('Volume (10^8 m^3)')
ax.set_xlim(1871, 1970)
plt.show()

### Data preprocessing
To make things easier for the model, we will normalise the year data to be between 0 and 1 and standardise the flow data to have a mean of 0 and a standard deviation of 1.

In [ ]:
volume = nile['volume'].values
year = nile['year'].values

# Standardise year
year_mean = year.mean()
year_std = year.std()
x = (year - year_mean) / year_std

# Standardise volume
volume_mean = volume.mean()
volume_std = volume.std()
y = (volume - volume_mean) / volume_std

print(x.shape, y.shape)

## Review of Hilbert Space approximate Gaussian Processes

Solin and Sarkka (2020) proposed to approximate stationary kernels as a truncated sum of the spectral density of the kernel evaluated at the square root of specific eigenvalues multiplied by the corresponding eigenvectors:
$$
k(x,x') \approx \sum_{m=1}^M S_\theta(\sqrt{\lambda_m}) \phi_m(x) \phi_m(x'),
$$
where $S_\theta$ is the spectral density of the kernel, $\lambda_m$ are the eigenvalues, and $\phi_m$ are the eigenvectors. The number of terms $M$ is a hyperparameter that controls the approximation accuracy.

Here, the eigenvalues and eigenvectors are given as,
$$
\lambda_m = \left( \frac{n\pi}{2L} \right)^2, \quad \phi_m(x) = \sqrt{\frac1L} \sin\left(\sqrt{\lambda_m}(x + L)\right).
$$
Here $L$ is a boundary condition which determines the size of the domain in which the GP is approximated.

The expression for the spectral density depends on the kernel. Here are 3 of the most commonly used stationary kernels and their spectral densities:

1. Squared exponential kernel:
$$
S_{\sigma,\ell}(\omega) = \sigma^2 \sqrt{2\pi} \ell \exp\left(-\frac{\ell^2\omega^2}{2}\right).
$$
2. Matern 3/2 kernel:
$$
S_{\sigma,\ell}(\omega) = \sigma^2 \frac{2\sqrt{3}}{\ell} \left(\frac{3}{\ell^2} + \omega^2 \right)^{-2}.
$$
3. Matern 5/2 kernel:
$$
S_{\sigma,\ell}(\omega) = \sigma^2 \frac{8\sqrt{5}}{3\ell} \left(\frac{5}{\ell^2} + \omega^2 \right)^{-5/2}.
$$

We can rewrite the truncated sum in matrix notation as
$$
k(x,x') \approx \mathbf{\phi}(x)^\top \mathbf{\Delta} \mathbf{\phi}(x'),
$$
where $\mathbf{\phi}(x) = \{ \phi_m(x) \}_{m=1}^M \in \mathbb{R}^M$ are the eigenvectors evaluated at the input point, and $\mathbf{\Delta} = \text{diag}(\{ S_\theta(\sqrt{\lambda_m}) \}_{m=1}^M) \in \mathbb{R}^{M \times M}$ is a diagonal matrix of the spectral density evaluated at the square root of the eigenvalues.

Then, the covariance matrix $\mathbf{K}$ can be approximated as
$$
\mathbf{K} \approx \tilde{\mathbf{K}} = \mathbf{\Phi} \mathbf{\Delta} \mathbf{\Phi}^\top,
$$
where
$$
\mathbf{\Phi} = \begin{pmatrix}
\phi_1(x_1) & \cdots & \phi_M(x_1) \\
\vdots & \ddots & \vdots \\
\phi_1(x_n) & \cdots & \phi_M(x_n)
\end{pmatrix}
$$
is a matrix of the eigenvectors. From this we have that the GP sample can be approximated as
$$
\mathbf{f} \approx \tilde{\mathbf{f}} \sim N(\mathbf{0}, \tilde{\mathbf{K}}).
$$
We can sample from the approximate GP by first sampling an auxiliary vector $\mathbf{z} \sim N(\mathbf{0}, \mathbf{I}_M)$ and then computing the approximate GP sample as
$$
\tilde{\mathbf{f}} = \mathbf{\Phi} \mathbf{\Delta}^{1/2} \mathbf{z}
$$
where $\mathbf{\Delta}^{1/2}$ is the square root of the diagonal matrix $\mathbf{\Delta}$:
$$
\mathbf{\Delta}^{1/2} = \begin{pmatrix}
S_\theta(\sqrt{\lambda_1})^{1/2} & & \\
& \ddots & \\
& & S_\theta(\sqrt{\lambda_M})^{1/2}
\end{pmatrix}.
$$

<details>
<summary>Click to see the proof</summary>

Let $\tilde{\mathbf{L}} = \mathbf{\Phi} \mathbf{\Delta}^{1/2}$. Then, we have that
$$
\tilde{\mathbf{K}} = \mathbf{\Phi}^\top \mathbf{\Delta} \mathbf{\Phi} = \tilde{\mathbf{L}} \tilde{\mathbf{L}}^\top.
$$
Let $\mathbf{z} \sim N(\mathbf{0}, \mathbf{I}_M)$. Then, we have that
$$
\mathbb{E}[\mathbf{z}] = \mathbf{0}
$$
and
$$
\mathbb{E}[\tilde{\mathbf{L}} \mathbf{z}] = \mathbf{\Phi} \mathbf{\Delta}^{1/2} \mathbb{E}[\mathbf{z}] = \mathbf{0}.
$$
Thus, $\tilde{\mathbf{L}} \mathbf{z}$ has zero mean. The covariance matrix of $\tilde{\mathbf{L}} \mathbf{z}$ is
$$
\text{Cov}(\tilde{\mathbf{L}} \mathbf{z})
= \mathbb{E}[(\tilde{\mathbf{L}} \mathbf{z}) (\tilde{\mathbf{L}}\mathbf{z})^\top] = \tilde{\mathbf{L}}\mathbb{E}[ \mathbf{z} \mathbf{z}^\top]\tilde{\mathbf{L}}^\top.
$$
Since $\mathbf{z} \sim N(\mathbf{0}, \mathbf{I}_M)$, we have that
$$
\mathbb{E}[\mathbf{z} \mathbf{z}^\top] = \mathbf{I}_M.
$$
Thus,
$$
\text{Cov}(\tilde{\mathbf{L}} \mathbf{z}) = \tilde{\mathbf{L}}\tilde{\mathbf{L}}^\top = \tilde{\mathbf{K}}.
$$
This confirms that $\tilde{\mathbf{f}} \sim N(\mathbf{0}, \tilde{\mathbf{K}})$ and concludes the proof.

</details>


### The boundary condition
HSGP is an approximation of a Gaussian process on a compact subspace of the real line $S \subset \mathbb{R}$. We are going to assume that our input points are centered around 0 and we are going to define our subspace as $[-L, L]$ where
$$
L = C \times \max(\mathbf{x}).
$$
We refer to $C$ as the boundary inflation factor and for this tutorial set $C = 1.5$.

## The model

Let $\mathbf{y} = (y_1,\ldots,y_n)^\top$ be a vector of outcomes, in this case flow volume. Let $\mathbf{x} = (x_1,\ldots,x_n)^\top$ be a vector of inputs, in this case year. We will model the data as
$$
\begin{align*}
\mathbf{y} &= \alpha + \tilde{f}(\mathbf{x}) + \boldsymbol{\varepsilon}, \\
\boldsymbol{\varepsilon} &\sim N(0, \sigma_{\varepsilon}^2\mathbf{I}_n), \\
\tilde{f}(\mathbf{x}) &\sim \text{HSGP}(\mathbf{x}; \mathbf{z}, \sigma, \ell), \\
\alpha &\sim N(0, 10), \\
\boldsymbol{z} &\sim N(0, \mathbf{I}_M), \\
\sigma &\sim \text{inv-Gamma}(5, 5) \\
\ell &\sim \text{inv-Gamma}(5, 5) \\
\sigma_{\varepsilon} &\sim \text{inv-Gamma}(5,5)
\end{align*}
$$


## Implementing HSGP in Stan

We will work in a new Stan file: call it `hsgp_regression.stan`. The Stan program will be similar in structure to the GP program, but with a few modifications. We will begin by implementing functions to construct the spectral densities, eigenvalues, and eigenvectors. We will put them together to define a functions named `hsgp_se`, `hsgp_matern32`, and `hsgp_matern52`, that take in the input points `x`, the hyperparameters `sigma` and `ell`, a matrix of eigenvectors `PHI`, and a vector of auxiliary variables `z`. The function will return the HSGP sample `f`.

### Spectral density functions

Let us start by defining the spectral densities. We will define three functions: `spd_se`, `spd_matern32`, and `spd_matern52`. These functions will take as input a vector of frequencies `omega` and the hyperparameters `sigma` and `ell`. The functions will return the spectral density evaluated at the frequencies.

```stan
functions {
	vector spd_se(vector omega, real sigma, real ell) {
		// Implement the spectral density for the squared exponential kernel
	}

	vector spd_matern32(vector omega, real sigma, real ell) {
		// Implement the spectral density for the Matern 3/2 kernel
	}

	vector spd_matern52(vector omega, real sigma, real ell) {
		// Implement the spectral density for the Matern 5/2 kernel
	}
}
```

### Eigenvalues and eigenvector functions
Next, we will implement functions to compute the eigenvalues and eigenvectors. We will define two functions: `eigenvalues` and `eigenvectors`. The `eigenvalues` function will take in the number basis functions `M` and the boundary condition `L` and return a vector of eigenvalues. The `eigenvectors` function will take in the input points `x`, the number of basis functions `M`, the boundary condition `L`, and the eigenvalues `lambda` and return a matrix of eigenvectors.

```stan
functions {
	// Other functions...

	vector eigenvalues(int M, real L) {
		// Implement the eigenvalues function
	}

	matrix eigenvectors(vector x, int M, real L, vector lambda) {
		// Implement the eigenvectors function
	}
}
```

### HSGP function
Finally, we put these components together to define the `hsgp` function. The function will take in the input points `x`, the hyperparameters `sigma` and `ell`, the eigenvector matrix `PHI`, and a vector of auxiliary variables `z`. The function will return a vector of the HSGP sample `f`.

```stan
functions {
	// Other functions...

	vector hsgp_se(vector x, real sigma, real ell, vector lambdas, matrix PHI, vector z) {
		int n = rows(x);
		int M = cols(PHI);
		vector[n] f;
		matrix[M, M] Delta;

		// Implement the HSGP function
		// 1. Compute the spectral densities

		// 2. Construct the diagonal matrix Delta

		// 3. Compute the HSGP sample

		return f;
	}

	vector hsgp_matern32(vector x, real sigma, real ell, matrix PHI, vector z) {
		// Implement the HSGP function
	}

	vector hsgp_matern52(vector x, real sigma, real ell, matrix PHI, vector z) {
		// Implement the HSGP function
	}
}
```

### Data block
We will need to define the data block to include the number of data points `N` (positive integer), the input points `x` (vector of length `N`), the outcome `y` (real valued array of length `N`), the boundary inflation constant `C` (positive real value), and the number of eigenfunctions `M` (positive integer).

### Transformed data block
In the transformed data block:
1. Define the boundary condition `L` as `C * max(x)`
2. Precompute the eigenvalues `lambda` and the eigenvectors `PHI` using the `eigenvalues` and `eigenvectors` functions.

### Parameters block
In the parameters block define:
1. The interecpt `alpha` (real value)
2. The noise standard deviation `sigma_eps` (positive real value)
3. vector of standard normal random variables `z`
4. The marginal GP variance `sigma` (positive real)
5. The GP lengthscale `ell` (positive real)

### Transformed parameters block
In the transformed parameters block, implement
1. `f`: a vector of size `N` which is an approximate sample of a GP with the squared exponential kernel.
2. `mu`: a vector of size `N` that contains the expected value of `y` at each input point.

### Model block
In the model block we assign priors to all the parameters defined in the parameters block and we define the likelihood.

### Generated quantities block
Finally, in the generated quatities block you compute the log likelihood `log_lik` for each data point as well as generate random samples `y_rep` based on the inferred parameters of the model.

Compile the Stan code using the `CmdStanModel` class from the `cmdstanpy` library.

In [ ]:
hsgp_se_model = CmdStanModel(stan_file='hsgp_regression.stan')

Create the stan data dictionary with the number of data points `N`, the input points `x`, the output points `y`, the number of basis functions `M`, the boundary inflation factor `C`, and the number of eigenvectors `M`.

Run the MCMC algorithm with 4 chains, 500 warmup iterations, and 1000 iterations. Set the `adapt_delta` argument to 0.95 and set the random seed.

In [ ]:
import time
start_time = time.time()

# ====================
# Your code here
# ====================

end_time = time.time()
runtime = end_time - start_time
print(f"Runtime of the Stan model: {runtime} seconds")

Use the `diagnose` method to perform a quick diagnosis of the model.

Define a dictionary `custom_summary_fns` of lambda functions that calculates the median, the 2.5% quantile and the 97.5% quantile.

Convert the fit object to an ArviZ inference data object using
the `from_cmdstanpy` function. Use the `summary` method and `custom_summary_fns`, summarise the posterior distribution of `mu` and `y_rep`

In [ ]:
idata_hsgp_se = az.from_cmdstanpy(hsgp_se_fit)

mu_hsgp_se_sum = az.summary(idata_hsgp_se,
                            var_names=['mu'],
                            stat_funcs=custom_summary_fns,
                            extend=False)

y_rep_hsgp_se_sum = az.summary(idata_hsgp_se,
                               var_names=['y_rep'],
                               stat_funcs=custom_summary_fns,
                               extend=False)

In [ ]:
# Function that plots the posterior summary against the observed data
def plot_posterior_summary(year, y, posterior_summary):
	fig, ax = plt.subplots()
	ax.scatter(year, y, label='Data', s=10)
	ax.plot(year, posterior_summary['median'], label='Posterior Median of f', color='red')
	ax.fill_between(year, posterior_summary['q2.5'], posterior_summary['q97.5'], color='gray', alpha=0.2, label='95% CI')
	ax.set_title('Posterior Median of f vs Data')
	ax.set_xlabel('Year')
	ax.set_ylabel('Volume (10^8 m^3)')
	ax.set_xlim(1871, 1970)
	ax.legend()
	plt.show()

In [ ]:
plot_posterior_summary(year, y, mu_hsgp_se_sum)

In [ ]:
plot_posterior_summary(year, y, y_rep_hsgp_se_sum)

Create a new Stan file titled `hsgp_matern32_regression.stan` where the squared exponential covariance kernel is replaced by the Matern 3/2 kernel.

In [ ]:
# Compile model

In [ ]:
# Run inference algorithm

In [ ]:
idata_hsgp_matern32 = az.from_cmdstanpy(hsgp_matern32_fit)

mu_hsgp_matern32_sum = az.summary(idata_hsgp_matern32,
                                  var_names=['mu'],
                                  stat_funcs=custom_summary_fns,
                                  extend=False)

y_rep_hsgp_matern32_sum = az.summary(idata_hsgp_matern32,
                                     var_names=['y_rep'],
                                     stat_funcs=custom_summary_fns,
                                     extend=False)

In [ ]:
plot_posterior_summary(year, y, mu_hsgp_matern32_sum)

In [ ]:
plot_posterior_summary(year, y, y_rep_hsgp_matern32_sum)

Create a new Stan file titled `hsgp_matern52_regression.stan` where the squared exponential covariance kernel is replaced by the Matern 5/2 kernel.

In [ ]:
# Compile model

In [ ]:
# Run inference algorithm

In [ ]:
idata_hsgp_matern52 = az.from_cmdstanpy(hsgp_matern52_fit)

mu_hsgp_matern52_sum = az.summary(idata_hsgp_matern52,
                                  var_names=['mu'],
                                  stat_funcs=custom_summary_fns,
                                  extend=False)

y_rep_hsgp_matern52_sum = az.summary(idata_hsgp_matern52,
                                     var_names=['y_rep'],
                                     stat_funcs=custom_summary_fns,
                                     extend=False)

In [ ]:
plot_posterior_summary(year, y, mu_hsgp_matern52_sum)

In [ ]:
plot_posterior_summary(year, y, y_rep_hsgp_matern52_sum)